## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/Daniel/Desktop/Module_Challenges/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.48,68,99,10.13,light rain
1,1,Chuy,UY,-33.6971,-53.4616,60.96,70,0,3.15,clear sky
2,2,Punta Arenas,CL,-53.1500,-70.9167,41.00,87,90,8.05,light rain
3,3,Kahului,US,20.8947,-156.4700,75.20,94,40,13.80,light rain
4,4,Santa Rosa,PH,14.3122,121.1114,82.40,78,20,3.44,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Santa Rosa,PH,14.3122,121.1114,82.40,78,20,3.44,few clouds
23,23,Puerto Ayora,EC,-0.7393,-90.3518,82.40,74,40,17.27,scattered clouds
24,24,Saint George,US,37.1041,-113.5841,82.99,12,1,8.05,clear sky
40,40,Hilo,US,19.7297,-155.0900,84.20,51,40,9.22,scattered clouds
46,46,Buchanan,LR,5.8808,-10.0467,80.60,89,20,7.34,few clouds
49,49,Muli,MV,2.9167,73.5667,83.05,73,5,5.82,clear sky
54,54,Hithadhoo,MV,-0.6000,73.0833,83.52,72,96,8.37,overcast clouds
65,65,Thayetmyo,MM,19.3167,95.1833,83.62,64,7,4.92,clear sky
75,75,Kalmunai,LK,7.4167,81.8167,80.19,79,71,0.67,broken clouds
76,76,Coahuayana,MX,18.7333,-103.6833,84.20,51,90,9.22,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                130
City                   130
Country                130
Lat                    130
Lng                    130
Max Temp               130
Humidity               130
Cloudiness             130
Wind Speed             130
Current Description    130
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = pd.DataFrame(preferred_cities_df)

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Santa Rosa,PH,82.40,few clouds,14.3122,121.1114,
23,Puerto Ayora,EC,82.40,scattered clouds,-0.7393,-90.3518,
24,Saint George,US,82.99,clear sky,37.1041,-113.5841,
40,Hilo,US,84.20,scattered clouds,19.7297,-155.0900,
46,Buchanan,LR,80.60,few clouds,5.8808,-10.0467,
49,Muli,MV,83.05,clear sky,2.9167,73.5667,
54,Hithadhoo,MV,83.52,overcast clouds,-0.6000,73.0833,
65,Thayetmyo,MM,83.62,clear sky,19.3167,95.1833,
75,Kalmunai,LK,80.19,broken clouds,7.4167,81.8167,
76,Coahuayana,MX,84.20,overcast clouds,18.7333,-103.6833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = pd.DataFrame(hotel_df)

In [10]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Santa Rosa,PH,82.40,few clouds,14.3122,121.1114,Erlyn’s Sta Rosa City Home
23,Puerto Ayora,EC,82.40,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
24,Saint George,US,82.99,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
40,Hilo,US,84.20,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
46,Buchanan,LR,80.60,few clouds,5.8808,-10.0467,Kamaneahn Hotel Inc
...,...,...,...,...,...,...,...
652,La Aldea,MX,89.01,clear sky,20.9000,-101.4833,HOTEL RESTER
656,Mitsamiouli,KM,81.14,few clouds,-11.3847,43.2844,Foyer ADM
660,Keti Bandar,PK,81.14,scattered clouds,24.1447,67.4497,Haji laloo memon house
662,Vaitape,PF,81.46,broken clouds,-16.5167,-151.7500,Conrad Bora Bora Nui


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/Daniel/Desktop/Module_Challenges/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))